### AI-Powered Research Assistant

#### Problem Statement
Traditional research workflows often face several limitations:
- Lack of depth in information gathering and analysis
- Inconsistent research methodology across different topics
- Limited context retention during complex research tasks
- Manual effort in synthesizing information from multiple sources
- Difficulty in maintaining research coherence across multiple subtopics

This notebook presents an innovative solution that overcomes these limitations by implementing an advanced research orchestration system.

#### Solution Architecture
Our implementation leverages a coordinated group of Large Language Models (LLMs), each specialized for different aspects of the research process:

1. **Research Planner (Deepseek)**: 
   - Analyzes research queries
   - Breaks down complex topics into structured subtasks
   - Creates detailed research execution plans

2. **Research Executor (Claude)**:
   - Executes the planned research steps
   - Manages tool interactions
   - Synthesizes information from multiple sources

3. **Information Retrieval (Perplexity AI)**:
   - Performs targeted web searches
   - Extracts relevant information
   - Maintains citation tracking

#### Current Implementation
This notebook implements the core research flow with:
- Multi-LLM orchestration
- Structured research planning
- Web-based information retrieval
- Citation management
- Research synthesis and reporting

#### Future Improvements
The system is designed to be enhanced with RAG (Retrieval-Augmented Generation) capabilities:
- Integration of domain-specific knowledge bases
- Custom vector stores for improved context retention
- Semantic search for better information retrieval
- Dynamic document chunking and embedding
- Hybrid search combining dense and sparse retrievers


In [ ]:
! pip install litellm openai anthropic python-dotenv

## Initialization

Import necessary libraries and configure API access. This implementation requires several API keys for different LLM providers, each serving a specific purpose in the research pipeline.

### Required API Keys
Configure these in your `.env` file:

```env
# Core LLM Providers
ANTHROPIC_API_KEY=      # For Claude models (tool execution)
OPENAI_API_KEY=         # For GPT models (optional alternative)
OPENROUTER_API_KEY=     # For accessing multiple models through a single endpoint
PERPLEXITYAI_API_KEY=   # For web search and information retrieval
DEEPSEEK_API_KEY=       # For planning and reasoning (if accessing directly)

In [ ]:
from openai import OpenAI
import anthropic
from dotenv import load_dotenv
import os
from litellm import completion
import re

load_dotenv()

#### System Prompts and Configuration
Define the core prompt template for the Deepseek model. This prompt instructs the model to:
1. Act as an expert researcher
2. Process complex research tasks
3. Create detailed analysis plans
4. Work with an LLM agent for plan execution

In [18]:
DEEPSEEK_MODEL = "openrouter/deepseek/deepseek-r1"

deepseek_prompt = """
You are an expert researcher with deep expertise in finance, legal, and tax matters.
The first input you will receive will be a complex Research task that needs to be carefully reasoned through to solve. 
Your task is to review the challenge, conduct thorough research, and create a detailed plan to analyze information, assess implications, and provide comprehensive insights.

You will have access to an LLM agent that is responsible for executing the plan that you create and will return results.

The LLM agent has access to the following functions:
    - search_web(Question)
        - This function performs a web search and returns relevant information based on the provided query
        
When creating a plan for the LLM to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - **Main actions are numbered** (e.g., 1, 2, 3).
    - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
        - **Sub-actions should start on new lines**
    - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the financial statement shows a profit, then...').
    - **For actions that require using one of the above functions defined**, write a step to call a function using backticks for the function name (e.g., `call the fetch_context function`).
        - Ensure that the proper input arguments are given to the model for instruction. There should not be any ambiguity in the inputs.
    - **The last step** in the instructions should always be calling the `instructions_complete` function. This is necessary so we know the LLM has completed all of the instructions you have given it.
    - **Detailed steps** The plan generated must be extremely detailed and thorough with explanations at every step.
Use markdown format when generating the plan with each step and sub-step.

Please find the scenario below.
"""

## Core Functions: Planning

The `call_planner` function serves as a flexible interface for generating structured research plans. It can be configured to work with various reasoning-focused language models:

### Model Options
- **Reasoning Models**:
  - Deepseek (current implementation)
  - o1-mini
  - qwen/qwq-32b-preview
  - gemini-2.0-flash-thinking-exp:free
  - Other models optimized for reasoning and planning

### API Integration Options
- **Direct API Access**:
  - OpenAI Client (currently used)
  - Anthropic Client
  - Custom API implementations

- **API Aggregators**:
  - OpenRouter
  - LiteLLM
  - Other model routing services

The function maintains a consistent interface regardless of the underlying model choice, making it easy to swap models based on:
- Cost considerations
- Performance requirements
- Availability in different regions
- Specific reasoning capabilities needed

The current implementation uses Deepseek through OpenAI, but the architecture is designed to be model-agnostic for maximum flexibility.

In [40]:
def call_planner(scenario):
    prompt = f"""
    {deepseek_prompt}
        
    Scenario:
    {scenario}

    Please provide the next steps in your plan.
    """
    client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"], base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[{'role': 'user', 'content': prompt}],
    )

    plan = response.choices[0].message.content
    return plan

#### Message Management
Helper function to manage and display various types of messages including:
- Status updates
- Research plans
- Assistant responses
- Function calls and their responses

In [20]:
def append_message(message_list, message):
    message_list.append(message)
    message_type = message.get('type', '')
    if message_type == 'status':
        print(message['message'])
    elif message_type == 'plan':
        print("\nPlan:\n", message['content'])
    elif message_type == 'assistant':
        print("\nAssistant:\n", message['content'])
    elif message_type == 'function_call':
        print(f"\nFunction call: {message['function_name']} with arguments {message['arguments']}")
    elif message_type == 'function_response':
        print(f"\nFunction response for {message['function_name']}: {message['response']}")
    else:
        print(message.get('content', ''))

#### Web Search Implementation
Implementation of the web search functionality using Perplexity AI API:
- Performs web searches with academic rigor
- Processes and formats citations
- Handles error cases gracefully

In [21]:
def search_web(query):
    """
    Function to search the web using Perplexity AI API and return the answer
    with inline citation links instead of numbered references.
    
    Args:
        query (str): The search query to be processed
        
    Returns:
        str: The response content with inline links replacing citations like [1], [2], etc.
    """
    client = OpenAI(api_key=os.environ["PERPLEXITYAI_API_KEY"], base_url="https://api.perplexity.ai")
    
    messages = [
        {
            "role": "system",
            "content": "You are an artificial intelligence assistant and you need to answer like a Data collection engine with as much information as possible.",
        },
        {   
            "role": "user",
            "content": query,
        },
    ]

    response = client.chat.completions.create(
        model="sonar",
        messages=messages,
    )
    
    try:
        response = client.chat.completions.create(
            model="sonar",
            messages=messages,
        )
        
        if not response.choices:
            logger.error("No response choices available")
            return "Error: No response received from the search"
            
        content = response.choices[0].message.content
        
        # Check if citations exist in the response
        citations = getattr(response, 'citations', None)
        
        # If no citations or citations is empty, return content as is
        if not citations:
            return content
            
        # Process citations if they exist
        def replace_citation_with_link(match):
            citation_num_str = match.group(0)[1:-1]
            try:
                citation_idx = int(citation_num_str) - 1
                if citation_idx < 0 or citation_idx >= len(citations):
                    return match.group(0)
                return f"({citations[citation_idx]})"
            except (ValueError, IndexError):
                return match.group(0)
        
        content_with_links = re.sub(r'\[\d+\]', replace_citation_with_link, content)
        return content_with_links
        
    except Exception as e:
        logger.error(f"Error during web search: {str(e)}")
        return f"Error during search: {str(e)}"


def instructions_complete(final_report):
    return f"Final Report: {final_report}"


function_mapping = {
    'search_web': search_web,
    'instructions_complete': instructions_complete
}

#### Tool Definitions
Define the available tools and their schemas for the LLM to use:
- search_web: For web-based research
- instructions_complete: For finalizing research tasks

In [22]:
TOOLS = [
    {
        "name": "search_web",
        "description": "Function performs a web search and returns relevant information based on the provided query",
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The search query to be processed"
                }
            },
            "required": ["query"],
        },
    },
    {
        "name": "instructions_complete",
        "description": "Function should be called when we have completed ALL of the instructions.",
        "input_schema": {
            "type": "object",
            "properties": {
                "final_report": {
                    "type": "string",
                    "description": "Final Report based on the analysis."
                }
            },
            "required": ["final_report"],
        }
    }
]

#### Assistant Configuration
Define the core behavior and responsibilities of the research assistant:
- Policy execution
- Decision-making process
- Action execution flow

In [24]:
system_prompt = """
You are a helpful assistant responsible for executing the policy on handling deep research Tasks. 
Your task is to follow the policy exactly as it is written and perform the necessary actions.

You must explain your decision-making process across various steps.

# Steps
1. **Read and Understand Policy**: Carefully read and fully understand the given policy on Deep research Task.
2. **Identify the exact step in the policy**: Determine which step in the policy you are at, and execute the instructions according to the policy.
3. **Decision Making**: Briefly explain your actions and why you are performing them.
4. **Action Execution**: Perform the actions required by calling any relevant functions and input parameters. 

POLICY:
"""

#### Response Processing
Helper function to parse and process responses from the Anthropic API:
- Extracts text content
- Identifies tool usage
- Manages response structure

In [25]:
def parse_response_blocks(response):
    """
    Parse the response from Anthropic API and identify text and tool blocks.
    Returns a tuple of (has_tool_block, text_content, tool_info)
    """
    content = response.content
    text_content = None
    tool_info = None
    has_tool_block = False

    for block in content:
        if hasattr(block, 'type'):
            if block.type == 'text':
                text_content = block.text
            elif block.type == 'tool_use':
                has_tool_block = True
                tool_info = {
                    'name': block.name,
                    'input': block.input,
                    'id': block.id
                }

    return has_tool_block, text_content, tool_info

#### Plan Execution
Core function that executes the generated research plan:
- Manages conversation flow with Claude
- Handles tool calls
- Processes responses

In [26]:
def plan_execute(message_list, plan):
    policy_prompt = system_prompt

    client = anthropic.Anthropic()
    messages = [
        {'role': 'user', 'content': plan}
    ]
    i = 1

    while True:
        response = client.messages.create(
                model="claude-3-5-haiku-20241022",
                max_tokens=1024,
                system=policy_prompt,
                tools=TOOLS,
                messages=messages
        )
        
        messages.append({"role": "assistant", "content": response.content})
    
        has_tool, text_content, tool_info = parse_response_blocks(response)
        print(f"Iteration-{i}\nHasTool = {has_tool}\ntext_content\n{text_content}\nTool_info = {tool_info}")
        
        if text_content is not None:
            append_message(message_list, {"type": "assistant", "content": text_content})
        
        if tool_info['name'] in function_mapping:
            res = function_mapping[tool_info['name']](**tool_info['input'])
            print(f"Tool {tool_info['name']} Response:\n", res)
            if tool_info['name'] == "instructions_complete":
                break
        else:
            print(f"Unknown tool: {tool_info['name']}")
            
        messages.append({"role": "user", "content": [{
            "type": "tool_result",
            "tool_use_id": tool_info['id'],
            "content": res
        }]})
        i += 1
    return messages
    

#### Scenario Processing
Main orchestration function that:
- Generates research plans
- Executes plans
- Manages message flow

In [36]:
def process_scenario(message_list, scenario):
    append_message(message_list, {'type': 'status', 'message': 'Generating plan...'})

    plan = call_planner(scenario)

    append_message(message_list, {'type': 'plan', 'content': plan})

    append_message(message_list, {'type': 'status', 'message': 'Executing plan...'})

    messages = plan_execute(message_list, plan)

    append_message(message_list, {'type': 'status', 'message': 'Processing complete.'})

    return messages

#### Research Outline Generation
Specialized function for creating structured research outlines:
- Analyzes scenarios
- Extracts key themes
- Generates hierarchical outlines

In [37]:
def generate_outline(messages, scenario):
    system_prompt = """
    You are a research assistant specialized in creating focused research outlines.
    Your role is to:
    1. Analyze the provided scenario/question
    2. Extract key research themes directly related to the question
    3. Generate a hierarchical outline covering only the relevant themes
    4. Use markdown format with clear heading levels (###, -, *)
    5. Ensure every heading directly addresses components of the research question

    Do not include:
    - Methodological steps (like data collection or verification)
    - Process-related headings (like "Final Steps")
    - Any content not directly answering the research question
    """
    
    user_prompt = f"""
    Based on the following research scenario and messages, create a focused outline that directly addresses the key components of the question.

    <Research>
        {scenario}
    </Research>
    
    <message>
        {messages}
    </message>

    REQUIREMENTS:
    1. Use markdown formatting
    2. Include only headings relevant to answering the scenario
    3. Structure as main headings (###) and subheadings (-)
    4. Ensure each heading connects to either:
    - Factors affecting Indian stock market performance
    - Investment approaches of experienced Indian investors

    OUTPUT FORMAT:
    ### [Main Topic]
    - [Subtopic]
    - [Subtopic]
    """
    
    response = completion(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}]
    )

    return response.choices[0].message.content

#### Final Report Generation
Specialized function for creating detailed, structured reports:
- Analyzes messages and research context
- Generates a report that precisely follows the given outline
- Provides in-depth analysis of research data

In [38]:
def get_final_report(messages, scenario, outline):
    system_prompt = """
    You are a professional report writer specializing in detailed research analysis and structured reporting.
    Your primary responsibility is to generate comprehensive reports that precisely follow given outline structures while providing in-depth analysis of research data.

    Key Responsibilities:
    - Strictly adhere to provided outline structures
    - Generate detailed content for each outline section and subsection
    - Ensure thorough coverage of all points in the outline
    - Maintain consistent depth of analysis across all sections
    - Support all findings with specific evidence from research data
    - Properly cite all sources using URLs in the format [Source](URL)
    - When referencing information, always include the relevant citation
    - Maintain a consistent citation style throughout the report
    """

    user_prompt = f"""
    Generate a comprehensive, detailed report following the exact structure of the provided outline.
    Each section must be thoroughly developed with supporting evidence from the research data.

    <Research Context>
        {scenario}
    </Research Context>

    <Research Structure>
        {outline}
    </Research Structure>

    <Research Data>
        {messages}
    </Research Data>

    Report Requirements:
    1. STRICTLY FOLLOW THE PROVIDED OUTLINE:
    - Generate content for every section and subsection in the outline
    - Maintain the exact hierarchy and organization specified
    - Use consistent heading levels that match the outline structure
    - Ensure no outline points are skipped or merged

    2. DETAIL AND EVIDENCE:
    - Provide extensive detail for each outline point
    - Include multiple supporting examples from research data
    - Quote relevant passages from the research materials
    - Analyze each point thoroughly before moving to the next

    3. FORMATTING AND STRUCTURE:
    - Use markdown headers that match outline levels (# for main sections, ## for subsections, etc.)
    - Include transitional text between sections to maintain flow
    - Format quotes and evidence appropriately
    - Maintain consistent depth across all sections

    4. ANALYSIS REQUIREMENTS:
    - Provide data-driven insights for each section
    - Include metrics and quantitative analysis where applicable
    - Draw connections between related findings
    - Support each conclusion with specific evidence and citations
    - When citing multiple sources for a claim, include all relevant URLs

    5. CITATION FORMAT:
    - Use markdown links for citations: [Source](URL)
    - Citations should be placed immediately after the relevant information
    - Multiple citations should be separated by commas
    - Ensure every URL is valid and properly formatted

    Generate the report now, beginning with the first outline point and maintaining strict adherence to the outline structure.
    Remember to include proper citations for all information and a complete references section at the end.
    """

    response = completion(
        model="openrouter/minimax/minimax-01",
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
    )               
    return response.choices[0].message.content

def clean_markdown(content):
    return content.replace('```markdown', '').replace('```', '')

#### Example Usage
Demonstration of the research system with a real-world scenario

In [ ]:
scenario = "What are the typical factors that affect the Indian stock market's performance, and what investment approaches do experienced investors generally follow in the Indian market context?"

messages = process_scenario([], scenario)

In [ ]:
outline = generate_outline(messages, scenario)
print(outline)

In [ ]:
final_report = get_final_report(messages, scenario, outline)
clean_final_report = clean_markdown(final_report)
print(clean_final_report)